# Setup

In [ ]:
! pip install datasets transformers torch
! pip install sentencepiece
! pip install accelerate -U

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch, sentencepiece
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available")

GPU available: Tesla T4


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Training

In [ ]:
def get_statement(text):
    return text.replace("[","").replace("]", "")

def make_dataset(file_name):
    df = pd.read_csv(file_name)

    df["original_statement"] = df["original_text"].apply(lambda x: get_statement(x))
    df["perturbed_statement"] = df["perturbed_text"].apply(lambda x: get_statement(x))

    df_og = df[["original_statement", "ground_truth_output"]]
    df_adv = df[["perturbed_statement", "ground_truth_output"]]
    df_og = df_og.rename(columns = {"original_statement": "statement", "ground_truth_output": "label"})
    df_adv = df_adv.rename(columns = {"perturbed_statement": "statement", "ground_truth_output": "label"})

    df_merged = pd.concat([df_og, df_adv], ignore_index=True)
    df_merged.reset_index(names="idx")

    train, validation = train_test_split(df_merged, test_size=0.2)


    train = Dataset.from_pandas(train)
    validation = Dataset.from_pandas(validation)
    my_adv_dataset = DatasetDict({
        "train": train,
        "validation": validation
    })
    return my_adv_dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["statement"], truncation=True, padding="max_length", max_length=128)

def load_model_and_tokenizer(save_directory):
    model = AutoModelForSequenceClassification.from_pretrained(save_directory)
    tokenizer = AutoTokenizer.from_pretrained(save_directory)
    return model, tokenizer

def train_model(model, tokenizer, dataset, adv_save_directory):
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    training_args = TrainingArguments(
        output_dir="./results/mrpc",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer
    )

    trainer.train()

    model.save_pretrained(adv_save_directory)
    tokenizer.save_pretrained(adv_save_directory)

# Evaluation

In [ ]:
def evaluate_model(model, tokenizer, validation_set, device):
    model.eval()
    preds = []

    with torch.no_grad():
        for item in validation_set:
            inputs = tokenizer(item['perturbed_statement'], return_tensors="pt", truncation=True, padding=True).to(device)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            preds.append(predictions.cpu().item())

    df = pd.DataFrame(validation_set)
    df["evaluation_output"] = preds
    return df

def evaluate_model_on_dataset(val_file, model_save_directory, log_save_file):
    df_val = pd.read_csv(val_file)
    df_val["perturbed_statement"] = df_val["perturbed_text"].apply(lambda x: get_statement(x))
    attacked_ds = DatasetDict({
        "validation": Dataset.from_pandas(df_val)
    })
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = AutoModelForSequenceClassification.from_pretrained(model_save_directory).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_save_directory)

    df = evaluate_model(model, tokenizer, attacked_ds["validation"], device)
    df.to_csv(log_save_file)
    model_perf = (df["ground_truth_output"] == df["evaluation_output"]).sum()
    return model_perf


# FFBERT

## TextBugger

In [ ]:
attack_name = "textbugger"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/trainfinlog_{attack_name}.csv"
save_directory = "mstafam/fine-tuned-bert-financial-sentiment-analysis"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/valfinlog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.384881
2,No log,0.308620


Number of correct outputs before training: 47 / 200
Number of correct outputs after training: 135 / 200


## TextFooler

In [ ]:
attack_name = "textfooler"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/trainfinlog_{attack_name}.csv"
save_directory = "mstafam/fine-tuned-bert-financial-sentiment-analysis"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/valfinlog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.378005
2,No log,0.418097


Number of correct outputs before training: 10 / 200
Number of correct outputs after training: 141 / 200


## DeepWordBug

In [ ]:
attack_name = "DeepWordBug"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/trainfinlog_{attack_name}.csv"
save_directory = "mstafam/fine-tuned-bert-financial-sentiment-analysis"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/valfinlog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.371580
2,No log,0.396986


Number of correct outputs before training: 29 / 200
Number of correct outputs after training: 125 / 200


## PWWS

In [ ]:
attack_name = "PWWS"

file_name = f"/content/drive/My Drive/CS6220_logs/Train_logs/trainfinlog_{attack_name}.csv"
save_directory = "mstafam/fine-tuned-bert-financial-sentiment-analysis"
adv_save_directory = f"/content/drive/My Drive/Adv_trained/{attack_name}/"

val_file = f"/content/drive/My Drive/CS6220_logs/Val_logs/valfinlog_{attack_name}.csv"
log_save_file_before = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_before.csv"
log_save_file_after = f"/content/drive/My Drive/CS6220_logs/Adv_logs/valfinlog_{attack_name}_after.csv"

In [ ]:
dataset = make_dataset(file_name)
model, tokenizer = load_model_and_tokenizer(save_directory)
train_model(model, tokenizer, dataset, adv_save_directory)
perf_before = evaluate_model_on_dataset(val_file, save_directory, log_save_file_before)
perf_after = evaluate_model_on_dataset(val_file, adv_save_directory, log_save_file_after)
print(f"Number of correct outputs before training: {perf_before} / 200")
print(f"Number of correct outputs after training: {perf_after} / 200")

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.267342
2,No log,0.360543


Number of correct outputs before training: 20 / 200
Number of correct outputs after training: 129 / 200


---------------------------------------------------------------

#Analytics: Number of quiries (successful)

In [ ]:
import pandas as pd

print('DeepWordBug:')
df = pd.read_csv('rte_vallog_DeepWordBug.csv')

num_queries_summary = df['num_queries'].describe()
successful_queries = df[df['result_type'] == 'Successful']
failed_queries = df[df['result_type'] == 'Failed']

successful_summary = successful_queries['num_queries'].describe()

failed_summary = failed_queries['num_queries'].describe()

# # Print the results
# print("Task 1: Without considering result_type")
# print(num_queries_summary)

print("\nTask 2: Considering result_type - Successful")
print(successful_summary)

# print("\nTask 2: Considering result_type - Failed")
# print(failed_summary)


In [ ]:
import pandas as pd

print('TextFooler:')
df = pd.read_csv('rte_vallog_textfooler.csv')

num_queries_summary = df['num_queries'].describe()

successful_queries = df[df['result_type'] == 'Successful']
failed_queries = df[df['result_type'] == 'Failed']


successful_summary = successful_queries['num_queries'].describe()
failed_summary = failed_queries['num_queries'].describe()

# Print the results
# print("Task 1: Without considering result_type")
# print(num_queries_summary)

print("\nTask 2: Considering result_type - Successful")
print(successful_summary)

# print("\nTask 2: Considering result_type - Failed")
# print(failed_summary)


TextFooler:

Task 2: Considering result_type - Successful
count    142.000000
mean     106.239437
std      112.614949
min       18.000000
25%       41.000000
50%       69.000000
75%      110.750000
max      668.000000
Name: num_queries, dtype: float64


In [ ]:
import pandas as pd

print('PWWS:')
df = pd.read_csv('rte_vallog_PWWS.csv')

num_queries_summary = df['num_queries'].describe()

successful_queries = df[df['result_type'] == 'Successful']
failed_queries = df[df['result_type'] == 'Failed']

successful_summary = successful_queries['num_queries'].describe()
failed_summary = failed_queries['num_queries'].describe()

# # Print the results
# print("Task 1: Without considering result_type")
# print(num_queries_summary)

print("\nTask 2: Considering result_type - Successful")
print(successful_summary)

# print("\nTask 2: Considering result_type - Failed")
# print(failed_summary)


PWWS:

Task 2: Considering result_type - Successful
count    137.000000
mean     294.321168
std      186.025533
min       49.000000
25%      152.000000
50%      244.000000
75%      374.000000
max      940.000000
Name: num_queries, dtype: float64


In [ ]:
import pandas as pd

print('textBugger:')
df = pd.read_csv('rte_vallog_textbugger.csv')

num_queries_summary = df['num_queries'].describe()

successful_queries = df[df['result_type'] == 'Successful']
failed_queries = df[df['result_type'] == 'Failed']

successful_summary = successful_queries['num_queries'].describe()

failed_summary = failed_queries['num_queries'].describe()

# Print the results
# print("Task 1: Without considering result_type")
# print(num_queries_summary)

print("\nTask 2: Considering result_type - Successful")
print(successful_summary)

# print("\nTask 2: Considering result_type - Failed")
# print(failed_summary)


textBugger:

Task 2: Considering result_type - Successful
count    133.000000
mean      63.639098
std       45.912027
min       21.000000
25%       36.000000
50%       47.000000
75%       75.000000
max      293.000000
Name: num_queries, dtype: float64
